### Load libraries

In [1]:
import os
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset,DataLoader
from tqdm.auto import tqdm
import torch.utils.data as data
import pandas as pd
import torch
import transformers

c:\Users\user\anaconda3\envs\V2X\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load tokenizer and model

In [2]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

c:\Users\user\anaconda3\envs\V2X\lib\site-packages\transformers\models\t5\tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


### Data processing

In [3]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="test.json")
body_text = dataset['train']['body']
input_text = []
for body in range(13762):
    input_text.append('summarize: ' + str(body_text[body]))


Found cached dataset json (C:/Users/user/.cache/huggingface/datasets/json/default-bb7f199b20d2363c/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 1/1 [00:00<00:00, 31.02it/s]


### Build Dataset

In [4]:
class HeadlineGenerationDataset(Dataset):
    def __init__(self, text, tokenizer, max_len = 512):
        self.data = []
        for t in text:
            input_t = tokenizer(t, truncation=True, padding="max_length", max_length=max_len)
            
                    
            self.data.append({'input_ids':torch.tensor(input_t['input_ids']),
                              'attention_mask':torch.tensor(input_t['attention_mask'])})

    def __getitem__(self, index):
        
         
        return self.data[index]
        

    def __len__(self):
        return len(self.data)

### Dataloader

In [5]:
test_set = HeadlineGenerationDataset(input_text, tokenizer,max_len = 512)
test_loader = DataLoader(test_set,batch_size = 6,shuffle = False, num_workers = 0, pin_memory = True)

### Test model

In [6]:
import os
import json

model.cuda()
model.load_state_dict(torch.load('model_best/pytorch_model.bin'))
model.eval()
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)
        for i in range(len(outputs)):
            output = tokenizer.decode(outputs[i], skip_special_tokens=True)
            print(output)
            print('------------------------')
            print('------------------------')
            print('------------------------')
            predicted_text = {"title": output}
            ans_json = json.dumps(predicted_text, separators=(',', ':'))
            with open('310581027.json', 'a') as f:
                f.write(ans_json + "\n")

  0%|          | 0/2294 [00:00<?, ?it/s]c:\Users\user\anaconda3\envs\V2X\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 1/2294 [00:02<1:46:18,  2.78s/it]

Loons announce international matchup with MX
------------------------
------------------------
------------------------
Kittens in Washington looking for their fur-ever homes
------------------------
------------------------
------------------------
Milwaukee Brewers stymied by Cincinnati Reds 7-1
------------------------
------------------------
------------------------
Flagler developer's'smart growth' project faces objections
------------------------
------------------------
------------------------
US Cannabinoid Markets To Soar, With US Leads The Way
------------------------
------------------------
------------------------
Nvidia (NVDA) Dips More Than Broader Markets: What You Should
------------------------
------------------------
------------------------


  0%|          | 2/2294 [00:03<55:07,  1.44s/it]  

Orioles introduce No. 1 pick Adley Rutschman to Baltimore
------------------------
------------------------
------------------------
Top Miami news: Prison guards knocked out inmate's teeth in gang
------------------------
------------------------
------------------------
Porsche Cayman 718 GT4 Clubsport: Porsche Cayman GT4 Trophy
------------------------
------------------------
------------------------
2019 NHL Draft: How to watch Round 1 on TV, online streaming
------------------------
------------------------
------------------------
Police investigating death in Lake Maggiore
------------------------
------------------------
------------------------
MacKenzie Lueck murder suspect wanted to build small secret room in basement
------------------------
------------------------
------------------------


  0%|          | 3/2294 [00:03<37:32,  1.02it/s]

...

AP-NORC survey: Americans give Trump mixed reviews for economic stewardship
------------------------
------------------------
------------------------
Sea Levels Could Reach a Million-Dollar High
------------------------
------------------------
------------------------
Mission Health hires Chad Patrick as new leader
------------------------
------------------------
------------------------
Did Kentucky Gov. Matt Bevin start a rap battle?
------------------------
------------------------
------------------------
Sandra Bullock reveals the reason she won her MTV award for Bird Box
------------------------
------------------------
------------------------
KUWTK Season 14 Recap: Kendall Jenner's Pepsi Commercial
------------------------
------------------------
------------------------


100%|█████████▉| 2293/2294 [16:08<00:00,  2.42it/s]

Gold Cup: Mexico vs Haiti, live stream, TV channel, live stream
------------------------
------------------------
------------------------
Here's how much $30,000 in unsubsidized federal student loans will cost you in debt
------------------------
------------------------
------------------------
Jessica Andrade and her husband robbed at gunpoint in Brazil: Report
------------------------
------------------------
------------------------
Report: Warriors, Knicks working on sign-and-trade for Quinn Cook
------------------------
------------------------
------------------------
Astronomers detect coronal mass ejection on star
------------------------
------------------------
------------------------
Alyssa Naeher finally emerges from Hope Solo's shadow with brilliant
------------------------
------------------------
------------------------


100%|██████████| 2294/2294 [16:08<00:00,  2.37it/s]

Chris Sale Identifies What Has Went Wrong In First Half Of Season
------------------------
------------------------
------------------------
Canada celebrates first World Series title
------------------------
------------------------
------------------------
Judge denies Flynn's request to withdraw from case
------------------------
------------------------
------------------------
John Milton: Conley, Worley have legendary duos
------------------------
------------------------
------------------------
